# Intro to postgreSQL and psycopg2

**Cameron Bronstein - _July, 2019_**



#### Dependencies

- `psycopg2`: version 2.7.7
- `postgreSQL`: [version 11.x](https://www.enterprisedb.com/downloads/postgres-postgresql-downloads)

#### Documentation
- [postgreSQL 11](https://www.postgresql.org/docs/11/index.html)

#### Data Sources
- [West Virginia Counties](https://www.westvirginia-demographics.com/counties_by_population)
- [Opioid Use Data](https://www.washingtonpost.com/graphics/2019/investigations/dea-pain-pill-database/?utm_term=.b945f1d2a341#download-resources)


In [ ]:
# in case psycopg2 is not installed...
# !pip install psycopg2-binary --user

In [4]:
# a few imports
import psycopg2
import pandas as pd

## Connecting to Postgres DB

**Arguments:**
- `host = localhost`
- `dbname` = **your choice of db**
- `user` = **your postgres username**

In [7]:
# establishin a connection
connection = psycopg2.connect(host = "localhost", dbname = "cambostein", user = "cambostein")

In [8]:
# creating the cursor - this is a python object that executes all of your SQL commands
cursor = connection.cursor()

### Create a table skeleton in the PG database

**Steps:**
- title
- column names
- column data types
- primary key

In [ ]:
command = """
CREATE TABLE west_virginia(
transaction_date date,
reporter_name varchar,
reporter_address varchar,
buyer_name varchar, 
buyer_category varchar,
buyer_address varchar,
buyer_county varchar,
product_name varchar,
drug_name varchar,
dosage_unit float,
quantity float,
base_wt_gm float,
labeler_name varchar,
company_name varchar)
"""

cursor.execute(command)

### Get the data

In [ ]:
# filling the empty table with data!
connection.commit()

with open("./data/west_virginia_for_lesson.tsv") as f:
    
    # why do we use next, here?
    next(f)
    
    # here is cursor, again
    cursor.copy_from(f, 'west_virginia', sep='\t')

# this establishes the end of a command block    
connection.commit()

In [13]:
# confirming that we have data

command = """
SELECT COUNT(*)
FROM west_virginia
"""

cursor.execute(command)

cursor.fetchall()

[(1879320,)]

### West Virgina Counties // Relational Databases 

In [14]:
counties = pd.read_csv('./data/west_virginia_pop_by_county.csv')
counties.head()

,County,Population,Rank
0,KANAWHA,180454,1
1,BERKELEY,117123,2
2,MONONGALIA,106420,3
3,CABELL,93224,4
4,WOOD,84203,5


In [15]:
counties.head()

,County,Population,Rank
0,KANAWHA,180454,1
1,BERKELEY,117123,2
2,MONONGALIA,106420,3
3,CABELL,93224,4
4,WOOD,84203,5


<details>
  <summary>How would we create a counties table?</summary>
  
  ```
  CREATE TABLE counties(
  County varchar,
  Population integer,
  Rank integer)
  
  with open("./data/west_virgina_pop_by_county.csv") as f:
      next(f)
    
      cursor.copy_from(f, 'counties', sep=',')

  connection.commit()
  ```
</details>

In [ ]:
connection.commit()

command = """
CREATE TABLE counties(
county varchar,
population integer,
rank integer)
"""

cursor.execute(command)

In [ ]:
# fill the counties table with data!
with open("./data/west_virgina_pop_by_county.csv") as f:
    next(f)

    cursor.copy_from(f, 'counties', sep=',')

connection.commit()

### SQL portion

- **First, an aside:**
    - Case study: [Leavenworth County, Kansas](https://www.kcur.org/post/leavenworth-county-kansas-may-not-be-catastrophic-opioid-hotspot-new-data-appear-show#stream/0). This is important to being a data scientist, too!

In [17]:
# dislapy the column names from the west_virginia table 


command = """
SELECT column_name, data_type 
FROM information_schema.columns
WHERE table_name = 'west_virginia';
"""

# this is the necessary syntax for psycopg2

cursor.execute(command)
cursor.fetchall()

[('transaction_date', 'date'),
 ('reporter_name', 'character varying'),
 ('reporter_address', 'character varying'),
 ('buyer_name', 'character varying'),
 ('buyer_category', 'character varying'),
 ('buyer_address', 'character varying'),
 ('buyer_county', 'character varying'),
 ('product_name', 'character varying'),
 ('drug_name', 'character varying'),
 ('dosage_unit', 'double precision'),
 ('quantity', 'double precision'),
 ('base_wt_gm', 'double precision'),
 ('labeler_name', 'character varying'),
 ('company_name', 'character varying')]

#### About psycopg2:

- **Take notice of the output of `cusor.fetchall()`**
    - It returns a list of tuples, with each element in the tuple ordered by the information specified in our query.
    - For the purpose of this lesson, this data format might not make sense, but it is important to understand as future Data Engineers and Data Scientists. This format might be more useful in an ETL (extract, transform, load) pipeline than running the data through a Pandas DataFrame
    
- For the rest of this lesson, we will run our queries through `pandas` as you are likely familiar with from previous lessons and assignments. 
    - **But** our connection to the `postgres` database still relies on `psycopg2`

### Query 1

<details>
  <summary>List the top 10 opioid buyers by grams purchased:</summary>
  
  ```
SELECT buyer_name as buyer,
       SUM(base_wt_gm) as total_ordered
FROM west_virginia
GROUP BY buyer_name
ORDER BY total_ordered DESC
LIMIT 10;
  ```
</details>

In [20]:
connection.commit()

command = """
SELECT buyer_name as buyer,
       SUM(base_wt_gm) as total_purchased
FROM west_virginia
GROUP BY buyer_name
ORDER BY total_purchased DESC
LIMIT 10;
"""

dataframe = pd.read_sql(command, connection)
dataframe

,buyer,total_purchased
0,"RITE AID OF WEST VIRGINIA, INC.",852078.017207
1,"WEST VIRGINIA CVS PHARMACY, L.L.C.",611592.554877
2,WALGREEN CO.,143850.924917
3,KROGER PHARMACY,119511.595889
4,FAMILY DISCOUNT PHARMACY INC,80820.607970
5,SAFESCRIPT PHARMACY #6,79154.099733
6,COLONY DRUG,76463.343661
7,STROSNIDER,75448.184850
8,"CRAB ORCHARD PHARMACY, INC.",65116.365655
9,WESTSIDE PHARMACY,63745.982688


### Query 2

<details>
  <summary>List the top 10 opioid buyers by number of transactions, compared to total grams ordered:</summary>
  
  ```
SELECT buyer_name as buyer,
       SUM(base_wt_gm) as total_ordered
       COUNT(buyer_name) as number_transactions
FROM west_virginia
GROUP BY buyer_name
ORDER BY number_transactions DESC
LIMIT 10;
  ```
</details>

In [22]:
connection.commit()

command = """
SELECT buyer_name as buyer,
       SUM(base_wt_gm) as total_purchased,
       COUNT(*) as number_transactions
FROM west_virginia
GROUP BY buyer_name
ORDER BY number_transactions DESC
LIMIT 10;
"""

dataframe = pd.read_sql(command, connection)
dataframe

,buyer,total_purchased,number_transactions
0,"RITE AID OF WEST VIRGINIA, INC.",852078.017207,279093
1,"WEST VIRGINIA CVS PHARMACY, L.L.C.",611592.554877,170254
2,KROGER PHARMACY,119511.595889,59640
3,WALGREEN CO.,143850.924917,30605
4,KMART PHARMACY,45872.402170,21902
5,FRUTH PHARMACY,43430.998939,19041
6,FRITZ'S PHARMACY AND WELLNESS,63583.640840,13997
7,DRUG EMPORIUM,48482.092400,13961
8,MEDICAP PHARMACY,45884.606621,13870
9,S & F PHARMACY,43398.957309,13301


### Query 3

<details>
  <summary>List the top 10 counties by number of transactions:</summary>
  
  ```
SELECT buyer_county as county,
       COUNT(buyer_county) as number_transactions
FROM west_virginia
GROUP BY county
ORDER BY number_transactions DESC
LIMIT 10;
  ```
</details>

In [24]:
connection.commit()

command = """
SELECT buyer_county as county,
       COUNT(*) as number_transactions
FROM west_virginia
GROUP BY county
ORDER BY number_transactions DESC
LIMIT 10;
"""

dataframe = pd.read_sql(command, connection)
dataframe

,county,number_transactions
0,KANAWHA,205127
1,CABELL,130426
2,RALEIGH,108253
3,WOOD,93235
4,HARRISON,84227
5,BERKELEY,81333
6,MONONGALIA,76353
7,MERCER,67443
8,LOGAN,65359
9,MARION,59912


### Query 4

<details>
  <summary>How does this compare to total number grams purchased per county?</summary>
  
  ```
SELECT buyer_county as county, 
       count(buyer_county) as number_transactions,
       SUM(base_wt_gm) as total_grams_purchased
FROM west_virginia
GROUP BY county
ORDER BY total_grams_purchased DESC
LIMIT 10;
  ```
</details>

In [25]:
connection.commit()

command = """
SELECT buyer_county as county, 
       count(buyer_county) as number_transactions,
       SUM(base_wt_gm) as total_grams_purchased
FROM west_virginia
GROUP BY county
ORDER BY total_grams_purchased DESC
LIMIT 10;
"""

dataframe = pd.read_sql(command, connection)
dataframe

,county,number_transactions,total_grams_purchased
0,KANAWHA,205127,590209.880169
1,CABELL,130426,503422.784330
2,RALEIGH,108253,444349.838749
3,BERKELEY,81333,309594.654079
4,LOGAN,65359,285758.722875
5,HARRISON,84227,258090.622130
6,MINGO,37154,228851.888449
7,WOOD,93235,201718.354192
8,MERCER,67443,199234.573589
9,GREENBRIER,54301,172816.286226


### Query 5

<details>
  <summary>What is the most common opioid product?</summary>
  
  ```
SELECT product_name as product,
       COUNT(product_name) as num_orders
FROM west_virginia
GROUP BY product
ORDER BY num_orders DESC
LIMIT 10;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 6

<details>
  <summary>How does opioid purchases compare to county population size?</summary>
  
  ```
SELECT w.buyer_county as county, 
       count(w.buyer_county) as number_transactions,
       SUM(w.base_wt_gm) as total_grams_purchased,
       c.Population as population
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
ORDER BY c.Population DESC
LIMIT 20;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 7

<details>
  <summary>How many counties have more opioid purchase transactions than people?</summary>
  
  ```
SELECT w.buyer_county as county, 
       count(w.buyer_county) as number_transactions,
       c.Population as population
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
HAVING count(w.buyer_county) > c.Population
ORDER BY number_transactions DESC
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 8

<details>
  <summary>What is their average number of opioids ordered by grams? Average number of pills?</summary>
  
  ```
SELECT w.buyer_county as county, 
       count(w.buyer_county) as number_transactions,
       c.Population as population,
       AVG(w.base_wt_gm) as average_grams_per_order,
       AVG(w.dosage_unit) as average_pills_per_order
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
HAVING count(w.buyer_county) > c.Population
ORDER BY average_grams_per_order DESC;
  ```
</details>

In [29]:
connection.commit()

command = """
SELECT w.buyer_county as county, 
       count(w.buyer_county) as number_transactions,
       c.Population as population,
       AVG(w.base_wt_gm) as average_grams_per_order,
       AVG(w.dosage_unit) as average_pills_per_order
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
HAVING count(w.buyer_county) > c.Population
ORDER BY average_grams_per_order DESC;
"""

dataframe = pd.read_sql(command, connection)
dataframe

,county,number_transactions,population,average_grams_per_order,average_pills_per_order
0,MINGO,37154,23785,6.159549,1030.027184
1,WYOMING,29596,20786,5.484123,605.195972
2,MCDOWELL,27310,18223,4.440139,557.447821
3,LOGAN,65359,32607,4.372140,697.483132
4,BOONE,28640,21951,4.299602,655.025838
5,RALEIGH,108253,74254,4.104735,472.946893
6,CABELL,130426,93224,3.859835,499.999770
7,MARSHALL,37991,30785,3.288899,465.712669
8,NICHOLAS,37456,24842,3.281076,484.961021
9,GREENBRIER,54301,34786,3.182562,414.733099


### Query 9

<details>
  <summary>What's the average number of pills per person per year?</summary>
  
  ```
SELECT w.buyer_county as county,
       c.Population as population,
       SUM(w.dosage_unit) as total_pills_purchased,
       SUM(w.dosage_unit) / 7 as pills_per_year,
       ((SUM(w.dosage_unit) / 7) / c.Population) as avg_pill_per_person 
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
ORDER BY avg_pill_per_person DESC;
  ```
</details>

In [28]:
connection.commit()

command = """
SELECT w.buyer_county as county,
       c.Population as population,
       SUM(w.dosage_unit) as total_pills_purchased,
       SUM(w.dosage_unit) / 7 as pills_per_year,
       ((SUM(w.dosage_unit) / 7) / c.Population) as avg_pill_per_person
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
ORDER BY avg_pill_per_person DESC
"""

dataframe = pd.read_sql(command, connection)
dataframe

,county,population,total_pills_purchased,pills_per_year,avg_pill_per_person
0,MINGO,23785,38269630.0,5.467090e+06,229.854530
1,LOGAN,32607,45586800.0,6.512400e+06,199.723986
2,WYOMING,20786,17911380.0,2.558769e+06,123.100576
3,BOONE,21951,18759940.0,2.679991e+06,122.089719
4,MCDOWELL,18223,15223900.0,2.174843e+06,119.346038
5,NICHOLAS,24842,18164700.0,2.594957e+06,104.458463
6,CABELL,93224,65212970.0,9.316139e+06,99.932835
7,RALEIGH,74254,51197920.0,7.313989e+06,98.499590
8,HANCOCK,29094,19682310.0,2.811759e+06,96.643932
9,GREENBRIER,34786,22520422.0,3.217203e+06,92.485573


### Query 10

<details>
  <summary>Pills per person - Statewide:</summary>
  
  ```
SELECT AVG(a.avg_pill_per_person) as statewide_average
FROM
(SELECT w.buyer_county as county,
       c.Population as population,
       SUM(w.dosage_unit) as total_pills_purchased,
       SUM(w.dosage_unit) / 7 as pills_per_year,
       ((SUM(w.dosage_unit) / 7) / c.Population) as avg_pill_per_person
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
ORDER BY avg_pill_per_person DESC)
AS a;
  ```
</details>

In [27]:
connection.commit()

command = """
SELECT AVG(a.avg_pill_per_person) as statewide_average
FROM
(SELECT w.buyer_county as county,
       c.Population as population,
       SUM(w.dosage_unit) as total_pills_purchased,
       SUM(w.dosage_unit) / 7 as pills_per_year,
       ((SUM(w.dosage_unit) / 7) / c.Population) as avg_pill_per_person
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
ORDER BY avg_pill_per_person DESC)
AS a;
"""

dataframe = pd.read_sql(command, connection)
dataframe

,statewide_average
0,62.509378


### Query 11 - Group By Date

<details>
  <summary>Lets group opioid purchases by year:</summary>
  
  ```
SELECT to_char(transaction_date, 'YYYY') as year,
       COUNT(w.*) as number_transactions,
       SUM(w.dosage_unit) as total_pills
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY year;
  ```
</details>

In [ ]:


connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

## What else can you think of?
- Which suppliers were most prominant? Per year?
- County averages per year? How do we show this?
    - Queries can get complicated pretty quick!

## Command Line Tricks

- end all queries with `;`
- `\timing` to print query execution times in CLI.
- `\d+` to print database information
- `\d+ table_name` to print table information
- `CREATE INDEX idx_name ON table_name (column_name);`
